In [50]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("Datasets/covid_toy.csv")
df.head(4)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No


In [4]:
df.shape

(100, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [6]:
df.gender.value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [7]:
df.city.value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [8]:
df.cough.value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=["has_covid"]),
                                                df["has_covid"],
                                                test_size=0.2,
                                                random_state=0)
x_train.shape, x_test.shape

((80, 5), (20, 5))

In [27]:
x_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


In [30]:
x_train.isnull().sum()

age       0
gender    0
fever     9
cough     0
city      0
dtype: int64

## Difficult Life : with separate encoders

In [31]:
# it will replace missing values with mean()
from sklearn.impute import SimpleImputer

si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[["fever"]])
x_test_fever = si.fit_transform(x_test[["fever"]])

In [37]:
x_train_fever.shape

(80, 1)

In [35]:
from sklearn.preprocessing import OrdinalEncoder

en = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = en.fit_transform(x_train[["cough"]])
x_test_cough = en.fit_transform(x_test[["cough"]])

In [38]:
x_train_cough.shape

(80, 1)

In [57]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop="first")
x_train_gender_city = ohe.fit_transform(x_train[["gender","city"]])
x_test_gender_city = ohe.fit_transform(x_test[["gender","city"]])

In [58]:
x_train_gender_city.shape

(80, 4)

In [59]:
# extracting age
x_train_age = x_train.drop(columns=['gender','cough','fever','city']).values
# also for test data
x_test_age = x_test.drop(columns=['gender','cough','fever','city']).values

x_train_age.shape

(80, 1)

In [60]:
x_train_new = np.concatenate((x_train_fever, x_train_gender_city, x_train_age, x_train_cough), axis=1)

# also for test data
x_test_new = np.concatenate((x_test_fever, x_test_gender_city, x_test_age, x_test_cough), axis=1)

In [62]:
x_train_new.shape

(80, 7)

## Simple Life : with column transformer

In [67]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OneHotEncoder(sparse_output=False,drop="first"), ['city','gender']),
    ('tnf3', OrdinalEncoder(categories=[["Mild","Strong"]]), ['cough'])
],remainder="passthrough")

In [68]:
x_train_transformer = transformer.fit_transform(x_train)

In [74]:
x_test_transformer = transformer.transform(x_test)

In [78]:
x_train_transformer.shape, x_test_transformer.shape

((80, 7), (20, 7))

In [77]:
x_test_transformer

array([[100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  19.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  25.        ],
       [101.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  42.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  81.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  69.        ],
       [ 98.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   1.        ,  34.        ],
       [ 99.        ,   0.        ,   0.        ,   1.        ,
          0.    